In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load ONNX model using OpenCV DNN
net = cv2.dnn.readNetFromONNX("best.onnx")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)   # Enable CUDA on Jetson
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Class labels (update if you have different classes)
class_names = ['herb paris', 'karela', 'small weed', 'grass', 'tori', 'horseweed', 'Bhindi', 'weed']

# Load and preprocess image
img_path = "images/1.jpg"
image = cv2.imread(img_path)
image_resized = cv2.resize(image, (640, 640))
blob = cv2.dnn.blobFromImage(image_resized, scalefactor=1/255.0, size=(640, 640), swapRB=True, crop=False)
net.setInput(blob)

# Run inference
outputs = net.forward()

# Postprocess
results = []
conf_thres = 0.25
nms_thres = 0.45

preds = outputs[0]  # shape: (25200, 85)
for pred in preds:
    cx, cy, w, h = pred[0:4]
    obj_conf = pred[4]
    class_probs = pred[5:]
    class_id = np.argmax(class_probs)
    class_score = class_probs[class_id]
    conf = obj_conf * class_score

    if conf > conf_thres:
        x = int((cx - w / 2) * image.shape[1] / 640)
        y = int((cy - h / 2) * image.shape[0] / 640)
        w = int(w * image.shape[1] / 640)
        h = int(h * image.shape[0] / 640)
        results.append([x, y, w, h, conf, class_id])

# Apply NMS
boxes = [r[:4] for r in results]
scores = [r[4] for r in results]
indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=conf_thres, nms_threshold=nms_thres)

# Draw results
for i in indices:
    i = i[0] if isinstance(i, (tuple, list, np.ndarray)) else i
    x, y, w, h, conf, cls_id = results[i]
    label = f"{class_names[cls_id]}: {conf:.2f}"
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

# Convert to RGB and show in notebook
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)
plt.axis('off')
plt.title("YOLOv5 Detections (OpenCV DNN + CUDA)")
plt.show()


ModuleNotFoundError: No module named 'onnxruntime'